In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt


In [2]:
import librosa
import soundfile as sf

In [3]:
from IPython.display import display, Audio

In [4]:
import librosa.display

In [5]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [6]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

In [7]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

In [8]:
import statistics

In [9]:
from recording_helper import record_audio, split_audio

In [10]:
input_voice = ["Shuffled_files_nine"]
# input_voice = ["filtered_split"]
# speaker_folders = ["Benjamin_Netanyau","Jens_Stoltenberg","Julia_Gillard","Magaret_Tarcher","Nelson_Mandela"]
speaker_folders = ["Benjamin_Netanyau_three","Jens_Stoltenberg_three","Julia_Gillard_three","Magaret_Tarcher_three","Nelson_Mandela_three"]

# number_of_files = 121

# input_path = "input_splits"
input_path = "D:\machine learning\speaker_recognition"
# output_path = "D:\machine learning\speaker_recognition\combined_files"
# shutil.rmtree(output_path, ignore_errors=True) # remove output folder if it exists
# os.makedirs(output_path, exist_ok  = True )

In [11]:
def extract_features(input_path, speaker_folders):
    features = []
    labels = []
    base = []
    for i, speaker_folder in enumerate(speaker_folders):
        speaker_folder_path = os.path.join(input_path, speaker_folder)
        print(speaker_folder_path)
        for file_name in os.listdir(speaker_folder_path):
            if file_name.endswith(".wav"):
                file_path = os.path.join(speaker_folder_path, file_name)
                audio, sr = librosa.load(file_path, sr=None, duration=1)
                base.append(file_path)
                # Extract MFCC features
                mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=32)  # Use n_mfcc=32
                
                # Normalize MFCC features
                mfccs = StandardScaler().fit_transform(mfccs)
                
                # Pad or truncate
                max_time_steps = 32
                max_mfcc_coeffs = 13
                if mfccs.shape[1] < max_time_steps:
                    mfccs = np.pad(mfccs, ((0, 0), (0, max_time_steps - mfccs.shape[1])))
                elif mfccs.shape[1] > max_time_steps:
                    mfccs = mfccs[:, :max_time_steps]
                
                if mfccs.shape[0] < max_mfcc_coeffs:
                    mfccs = np.pad(mfccs, ((0, max_mfcc_coeffs - mfccs.shape[0]), (0, 0)))
                elif mfccs.shape[0] > max_mfcc_coeffs:
                    mfccs = mfccs[:max_mfcc_coeffs, :]
                    
                features.append(mfccs.T)  # Transpose to have (time_steps, n_mfcc)
                labels.append(i)
                
    return np.array(features), np.array(labels), np.array(base)

In [12]:
x, y, b = extract_features(input_path, input_voice)

D:\machine learning\speaker_recognition\Shuffled_files_nine


In [15]:
model = tf.keras.models.load_model("path_to_save_model_nine.h5")


#evaluate the model on test set
y_pred_probabilities = model.predict(x)
y_pred = np.argmax(y_pred_probabilities, axis=1)
res = statistics.mode(y_pred)
print(speaker_folders[res])

#creating a confusion matrix
conf_matrix = confusion_matrix(x, y_pred, labels = speaker_folders)


3/3 [==============================] - 1s 9ms/step
Magaret_Tarcher_three


ValueError: Classification metrics can't handle a mix of unknown and multiclass targets

In [ ]:
if res == 0:
    # os.remove(recording_file)
    print("No voice recorded")
else :
    print("Voice recorded successfully")

Voice recorded successfully


In [ ]:
print(len(y_pred))
print(y_pred)

84
[1 0 2 3 4 3 1 1 2 1 0 2 2 0 4 3 0 2 0 1 1 2 3 0 1 4 3 1 4 3 2 2 3 3 1 3 2
 3 4 4 0 4 4 0 1 1 1 1 2 3 1 3 4 2 3 4 2 1 4 2 3 3 3 4 3 3 3 2 3 2 0 4 0 4
 0 0 4 4 0 0 4 0 0 3]


In [ ]:
filtered_b = b[y_pred != 0]

In [ ]:

# for i in filtered_b:
#     print(i)

In [ ]:
# for i in b:
#     print(i)

In [ ]:
import os
import librosa
import soundfile as sf



# Define output file name and path
output_file_name = "result_combined_nine.wav"
output_folder = r"D:\machine learning\speaker_recognition\result_combined"
os.makedirs(output_folder, exist_ok=True)

# Initialize combined audio signal
combined_audio = None
sr = None  # Sample rate

# Load and concatenate audio files
for file_path in filtered_b:
    # Load audio file
    audio, sr = librosa.load(file_path, sr=None)
    
    # Initialize combined audio signal if not already initialized
    if combined_audio is None:
        combined_audio = audio
    else:
        # Concatenate audio data
        combined_audio = np.concatenate((combined_audio, audio), axis=0)

# Save the combined audio data as a new audio file
output_file_path = os.path.join(output_folder, output_file_name)
sf.write(output_file_path, combined_audio, sr)


In [ ]:
out_voice = []
